In [56]:
import cv2
import torch
from os import listdir
from os.path import isfile, join
import pandas as pd

# Model
model = torch.hub.load("ultralytics/yolov5", "yolov5s", pretrained=True)


def load_images(dataset, label):
    imgs = [(dataset + f, label) for f in listdir(dataset) if isfile(join(dataset, f))]
    return imgs


imgs = load_images("../../data/clean_sink/", 0)
imgs += load_images("../../data/dirty_sink/", 1)

# imgs = imgs[:10]
# Inference
results = model(
    [img[0] for img in imgs],
)

# Results
# results.print()
results.save()  # or .show()


interested = ["bowl", "cup", "spoon", "knife", "fork"]


tp = fp = fn = tn = 0


def check_intersecting(box1, box2):
    intersection = 0
    dx = min(box1["xmax"], box2["xmax"]) - max(box1["xmin"], box2["xmin"])
    dy = min(box1["ymax"], box2["ymax"]) - max(box1["ymin"], box2["ymin"])
    if (dx >= 0) and (dy >= 0):
        intersection = dx * dy
    area = (box2["ymax"] - box2["ymin"]) * (box2["xmax"] - box2["xmin"])
    # print(intersection, area)
    return intersection > 0.8 * area



def classify_image(img_path, df):
    global tp, fp, fn, tn
    image = cv2.imread(img_path[0])
    h, w, c = image.shape

    dirty = df["name"].isin(interested).any()

    if "sink" in df["name"].values:
        sink = df[df["name"] == "sink"].iloc[0]
        items = df[df["name"].isin(interested)]
        dirty = False
        for i in range(len(items)):
            if check_intersecting(sink, items.iloc[i]):
                dirty = True
                break

    if dirty and img_path[1] == 1:
        tp += 1
    elif dirty and img_path[1] == 0:
        print(img_path[0])
        fp += 1
    elif not dirty and img_path[1] == 1:
        print(img_path[0])
        fn += 1
    else:
        tn += 1


for i in range(len(imgs)):
    classify_image(imgs[i], results.pandas().xyxy[i])

print("True Positives: ", tp)
print("False Positives: ", fp)
print("False Negatives: ", fn)
print("True Negatives: ", tn)


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0', 'setuptools>=65.5.1', 'wheel>=0.38.0'] not found, attempting AutoUpdate...


Using cache found in /Users/jkong/.cache/torch/hub/ultralytics_yolov5_master



requirements: AutoUpdate success ✅ 2.8s, installed 4 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0', 'setuptools>=65.5.1', 'wheel>=0.38.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-5-13 Python-3.9.6 torch-2.3.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
Saved 169 images to runs/detect/exp3


../../data/clean_sink/clean-57.webp
../../data/clean_sink/clean-11.jpg
../../data/clean_sink/clean-14.jpg
../../data/clean_sink/clean-84.webp
../../data/clean_sink/clean-8.jpg
../../data/clean_sink/clean-38.webp
../../data/clean_sink/clean-32.jpg
../../data/clean_sink/clean-25.jpg
../../data/clean_sink/clean-21.jpg
../../data/clean_sink/clean-78.webp
../../data/dirty_sink/dirty-42.jpg
../../data/dirty_sink/dirty-54.jpg
../../data/dirty_sink/dirty-41.jpg
../../data/dirty_sink/dirty-55.jpg
../../data/dirty_sink/dirty-5.jpg
../../data/dirty_sink/dirty-44.jpg
../../data/dirty_sink/dirty-46.jpg
../../data/dirty_sink/dirty-7.jpg
../../data/dirty_sink/dirty-26.jpg
../../data/dirty_sink/dirty-27.jpg
../../data/dirty_sink/dirty-15.jpg
../../data/dirty_sink/dirty-29.jpg
../../data/dirty_sink/dirty-28.jpg
../../data/dirty_sink/dirty-38.jpg
../../data/dirty_sink/dirty-11.jpg
../../data/dirty_sink/dirty-39.jpg
../../data/dirty_sink/dirty-9.jpg
../../data/dirty_sink/dirty-8.jpg
../../data/dirty_sink

In [30]:
results.pandas().xywh[0]

,xcenter,ycenter,width,height,confidence,class,name
0,156.785248,230.262955,113.970596,228.231598,0.819342,39,bottle
1,537.250916,411.521210,1058.515015,787.255676,0.367338,71,sink
2,555.295471,166.990509,89.575531,333.981018,0.361389,44,spoon


In [41]:
results.tolist()[0]

YOLOv5 <class 'models.common.Detections'> instance
image 1/1: 810x1080 1 bottle, 1 spoon, 1 sink
Speed: 2509.3ms pre-process, 36833.3ms inference, 112.4ms NMS per image at shape (169, 3, 640, 640)

In [55]:
# image = cv2.imread(img_path[0])
# h, w, c = image.shape

for i, image in enumerate(results.tolist()): 
    image = cv2.imread(imgs[i])
    h,w,c = image.shape
    print(h, w, c)

TypeError: Can't convert object to 'str' for 'filename'

In [58]:
def load_images(dataset, label):
    imgs = [(dataset + f, label) for f in listdir(dataset) if isfile(join(dataset, f))]
    return imgs


imgs = load_images("../../data/clean_sink/", 0)
imgs += load_images("../../data/dirty_sink/", 1)

# imgs = imgs[:10]
# Inference
results = model(
    [img[0] for img in imgs],
)

In [144]:
interested = ["bowl", "cup", "spoon", "knife", "fork"]

def get_items(features_detected: str) -> bool:
    if "(no detections)" in features_detected: 
        return []
    words = features_detected.split()[3:]
    items = []
    count = 0
    
    for word in words: 
        if word != "Speed:": 
            items.append(word)
            try: 
                _ = int(word)
                count += 1 
            except: 
                pass
        else: 
            break
    return items, count



df = pd.DataFrame(results.tolist())
df['label'] = ["clean" in file_str for file_str in results.files]
df['no_items'] = ['(no detections)' in str(lst) for lst in df[0]]
df['items_list'] = [get_items(str(lst)) for lst in df[0]]



xmin0 = []
ymin0 = []
xmax0 = []
ymax0 = []
conf0 = []

for i in range(len(df)): 
    if len(df['items_list'].iloc[i]) > 0: 
        xmin0.append(results.pandas().xyxy[i]['xmin'].iloc[0])
        ymin0.append(results.pandas().xyxy[i]['ymin'].iloc[0])
        xmax0.append(results.pandas().xyxy[i]['xmax'].iloc[0])
        ymax0.append(results.pandas().xyxy[i]['ymax'].iloc[0])
        conf0.append(results.pandas().xyxy[i]['confidence'].iloc[0])
        # df['xmin1'].iloc[i] = results.pandas().xyxy[i]['xmin'].iloc[1]
        # df['ymin1'].iloc[i] = results.pandas().xyxy[i]['ymin'].iloc[1]
        # df['xmax1'].iloc[i] = results.pandas().xyxy[i]['xmax'].iloc[1]
        # df['ymax1'].iloc[i] = results.pandas().xyxy[i]['ymax'].iloc[1]
        # df['conf'].iloc[i] = results.pandas().xyxy[i]['confidence'].iloc[1]
        # df['xmin2'].iloc[i] = results.pandas().xyxy[i]['xmin'].iloc[2]
        # df['ymin2'].iloc[i] = results.pandas().xyxy[i]['ymin'].iloc[2]
        # df['xmax2'].iloc[i] = results.pandas().xyxy[i]['xmax'].iloc[2]
        # df['ymax2'].iloc[i] = results.pandas().xyxy[i]['ymax'].iloc[2]
        # df['conf'].iloc[i] = results.pandas().xyxy[i]['confidence'].iloc[2]
    else: 
        xmin0.append(0)
        ymin0.append(0)
        xmax0.append(0)
        ymax0.append(0)
        dconf.append(0)
    

df['xmin0'] = xmin0
df['ymin0'] = ymin0
df['xmax0'] = xmax0
df['ymax0'] = ymax0
df['conf0'] = dconf
df

,0,label,no_items,items_list,xmin0,ymin0,xmax0,ymax0,conf
0,"image 1/1: 810x1080 1 bottle, 1 spoon, 1 sink\...",True,False,"([1, bottle,, 1, spoon,, 1, sink], 3)",99.799942,116.147148,213.770538,344.378754,0.819342
1,"image 1/1: 1920x1080 1 bottle, 1 sink, 1 tooth...",True,False,"([1, bottle,, 1, sink,, 1, toothbrush], 3)",11.150253,834.737732,1047.180298,1709.684448,0.545835
2,image 1/1: 225x225 (no detections)\nSpeed: 255...,True,True,[],0.000000,0.000000,0.000000,0.000000,0.000000
3,"image 1/1: 2000x2000 3 wine glasss, 2 knifes, ...",True,False,"([3, wine, glasss,, 2, knifes,, 4, apples,, 2,...",473.049835,199.024200,637.472473,568.286926,0.738120
4,image 1/1: 183x275 (no detections)\nSpeed: 255...,True,True,[],0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
164,"image 1/1: 683x1024 2 bottles, 1 cup, 2 sinks\...",False,False,"([2, bottles,, 1, cup,, 2, sinks], 3)",744.160156,129.359695,833.078308,304.637848,0.816222
165,"image 1/1: 473x700 3 cups, 1 fork, 1 knife, 2 ...",False,False,"([3, cups,, 1, fork,, 1, knife,, 2, spoons,, 1...",292.862152,215.578598,334.221313,300.143616,0.675537
166,"image 1/1: 1157x1600 1 spoon, 1 toilet\nSpeed:...",False,False,"([1, spoon,, 1, toilet], 2)",185.802383,145.804153,630.513062,483.068542,0.768076
167,image 1/1: 225x225 (no detections)\nSpeed: 255...,False,True,[],0.000000,0.000000,0.000000,0.000000,0.000000


In [135]:
res=[]
for result in results.tolist(): 
    res.append(len(get_items(str(result))))
print(max(res))

19


In [126]:
results.pandas().xyxy[0]

,xmin,ymin,xmax,ymax,confidence,class,name
0,99.799942,116.147148,213.770538,344.378754,0.819342,39,bottle
1,7.993378,17.893354,1066.508423,805.149048,0.367338,71,sink
2,510.507721,0.000000,600.083252,333.981018,0.361389,44,spoon
